# Few-shot prompting

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [2]:
def get_response(prompt):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}], 
    temperature = 0)
  return response.choices[0].message.content

## Controlling output structure

In [3]:
prompt = """
Q: Extract the odd numbers from {1, 3, 7, 12, 19}. A: {1, 3, 7, 19}
Q: Extract the odd numbers from {3, 5, 11, 12, 16}. A:
"""

response = get_response(prompt)
print(response)

{3, 5, 11}


## Sentiment analysis with few-shot prompting

In [4]:
response = client.chat.completions.create(
  model = "gpt-3.5-turbo",
  messages = [{"role": "user", "content": "The product quality exceeded my expectations."},
              {"role": "assistant", "content": "1"},
              {"role": "user", "content": "I had a terrible experience with this product's customer service."},
              {"role": "assistant", "content": "-1"},
              {"role": "user", "content": "The price of the product is really fair given its features."}
             ],
  temperature = 0
)
print(response.choices[0].message.content)

1


# Multi-step prompting

## Single-step prompt

In [5]:
prompt = "Create a plan for a beach vacation."

response = get_response(prompt)
print(response)

Plan for a Beach Vacation:

1. Choose the Destination:
   - Research different beach destinations based on preferences such as tropical, secluded, family-friendly, or adventurous.
   - Consider factors like weather, accessibility, and budget.
   - Popular options include Hawaii, Maldives, Bali, Cancun, or the Greek Islands.

2. Set a Budget:
   - Determine the overall budget for the vacation, including accommodation, transportation, meals, activities, and souvenirs.
   - Allocate funds for each category to ensure a balanced spending plan.

3. Book Accommodation:
   - Look for beachfront resorts, hotels, or vacation rentals that suit your preferences and budget.
   - Consider amenities like pools, beach access, restaurants, and proximity to attractions.
   - Read reviews and compare prices before making a reservation.

4. Arrange Transportation:
   - Decide on the mode of transportation, whether it's flying, driving, or taking a train.
   - Book flights or plan the road trip accordingly

## Multi-step prompt

In [6]:
prompt = """Create a plan for a beach vacation as follows:
Step 1 - Specify four potential locations.
Step 2 - Offer some accommodation options in each location.
Step 3 - Offer some activities in each location.
Step 4 - Do an evaluation of the pros and cons of each location."""

response = get_response(prompt)
print(response)

Step 1 - Specify four potential locations:
1. Maui, Hawaii
2. Cancun, Mexico
3. Bali, Indonesia
4. Maldives

Step 2 - Accommodation options in each location:
Maui, Hawaii:
- The Ritz-Carlton, Kapalua
- Four Seasons Resort Maui at Wailea
- Grand Wailea Maui, A Waldorf Astoria Resort

Cancun, Mexico:
- JW Marriott Cancun Resort & Spa
- The Ritz-Carlton, Cancun
- Hyatt Ziva Cancun

Bali, Indonesia:
- Ayana Resort and Spa, Bali
- The Mulia Bali
- COMO Uma Canggu

Maldives:
- Gili Lankanfushi Maldives
- Six Senses Laamu
- Conrad Maldives Rangali Island

Step 3 - Activities in each location:
Maui, Hawaii:
- Snorkeling at Molokini Crater
- Road to Hana scenic drive
- Surfing lessons at Ka'anapali Beach

Cancun, Mexico:
- Exploring the Mayan ruins of Tulum
- Swimming in the cenotes (natural sinkholes)
- Scuba diving in the Mesoamerican Barrier Reef

Bali, Indonesia:
- Visiting the iconic Uluwatu Temple
- Exploring the rice terraces in Ubud
- Taking a traditional Balinese cooking class

Maldive

## Analyze solution correctness

In [7]:
code = '''
def calculate_rectangle_area(length, width):
    area = length * width
    return area
'''

prompt = f"""
     Analyze the correctness of the function delimited by triple backticks according to the following criteria:
      1- It should have correct syntax
      2- The function should receive only 2 inputs
      3- The function should return only one output
      ```{code}```
    """

response = get_response(prompt)
print(response)

The function meets all the criteria:

1- It has correct syntax. The function is defined using the `def` keyword, followed by the function name `calculate_rectangle_area`, and the parameters `length` and `width` are specified within the parentheses. The function body is indented correctly, and the `return` statement is used to return the calculated area.

2- The function receives only 2 inputs. The parameters `length` and `width` are specified in the function definition, indicating that the function expects these two inputs.

3- The function returns only one output. The `return` statement is used to return the calculated area, which is stored in the `area` variable. Since only one value is returned, the function meets this criterion.

Overall, the function appears to be correct according to the given criteria.


# Chain-of-thought and self-consistency prompting

## Reasoning with chain-of-thought prompts

In [8]:
prompt = """My friend's father is double my friend's age, and my friend is 20. What will be my friend's father age in 10 years?
A: Let's think step by step"""

response = get_response(prompt)
print(response)

First, we need to determine the age of the friend's father. We know that the friend's father is double the age of the friend, who is 20 years old. 

So, the friend's father's age is 20 x 2 = 40 years old.

Now, we need to calculate the friend's father's age in 10 years. 

In 10 years, the friend's father will be 40 + 10 = 50 years old. 

Therefore, in 10 years, the friend's father will be 50 years old.


## One-shot chain-of-thought prompts

In [9]:
example = """Q: Sum the even numbers in the following set: {9, 10, 13, 4, 2}.
             A: Even numbers: {10, 4, 2}. Adding them: 10+4+2=16"""

question = """Q: Sum the even numbers in the following set: {15, 13, 82, 7, 14}. 
              A:"""

prompt = example + question
response = get_response(prompt)
print(response)

Even numbers: {82, 14}. Adding them: 82+14=96


## Self-consistency prompts

In [10]:
self_consistency_instruction = """Imagine three completely independent experts who reason differently are answering this question. The final answer is obtained by majority vote. The question is: """

problem_to_solve = "If you own a store that sells laptops and mobile phones. You start your day with 50 devices in the store, out of which 60% are mobile phones. Throughout the day, three clients visited the store, each of them bought one mobile phone, and one of them bought additionally a laptop. Also, you added to your collection 10 laptops and 5 mobile phones. How many laptops and mobile phones do you have by the end of the day?"

prompt = self_consistency_instruction + problem_to_solve

response = get_response(prompt)
print(response)

Expert 1: 

At the start of the day, we have 50 devices in total, out of which 60% are mobile phones. So, we have 50 * 0.6 = 30 mobile phones and the remaining 50 - 30 = 20 laptops.

Throughout the day, three clients visited the store, each buying one mobile phone. So, we sell 3 mobile phones, which leaves us with 30 - 3 = 27 mobile phones.

One of the clients also bought a laptop, so we sell one laptop, leaving us with 20 - 1 = 19 laptops.

We added 10 laptops and 5 mobile phones to our collection. So, we have 19 + 10 = 29 laptops and 27 + 5 = 32 mobile phones.

Therefore, by the end of the day, we have 29 laptops and 32 mobile phones.

Expert 2: 

At the start of the day, we have 50 devices in total, out of which 60% are mobile phones. So, we have 50 * 0.6 = 30 mobile phones and the remaining 50 - 30 = 20 laptops.

Throughout the day, three clients visited the store, each buying one mobile phone. So, we sell 3 mobile phones, which leaves us with 30 - 3 = 27 mobile phones.

One of the

# Iterative prompt engineering and refinement

## Iterative prompt engineering for standard prompts

In [11]:
#prompt = "Give me the top 10 pre-trained language models"
#prompt = "Give me a table with the top 10 pre-trained language models"
prompt = "Give me a table with the top 10 pre-trained language models with the following columns: model name, release year and owning company"

response = get_response(prompt)
print(response)

Here is a table with the top 10 pre-trained language models, including their model name, release year, and owning company:

| Model Name              | Release Year | Owning Company |
|-------------------------|--------------|----------------|
| GPT-3                   | 2020         | OpenAI         |
| BERT                    | 2018         | Google         |
| GPT-2                   | 2019         | OpenAI         |
| RoBERTa                 | 2019         | Facebook       |
| T5                      | 2020         | Google         |
| XLNet                   | 2019         | Google         |
| ALBERT                  | 2019         | Google         |
| ELECTRA                 | 2020         | Google         |
| CTRL                    | 2019         | OpenAI         |
| DistilBERT              | 2019         | Hugging Face   |

Please note that this list is not exhaustive, and there are other notable pre-trained language models available as well.


## Iterative prompt engineering fo few-shot prompts

In [12]:
prompt = """
Receiving a promotion at work made me feel on top of the world -> Happiness
The movie's ending left me with a heavy feeling in my chest -> Sadness
Walking alone in the dark alley sent shivers down my spine -> Fear
Time goes by so slowly -> no explicit emotion
Time flies like an arrow ->
"""

response = get_response(prompt)
print(response)

no explicit emotion
